In [22]:
import time
import datetime
import re
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [29]:
def writeLog(log_text, function_name):
    with open('web_parcer.log', 'a') as file:
        file.write(str(datetime.datetime.now()) + ":\t" + function_name + ': \t' + log_text + '\n')


def goToPage(query, driver):
    start_loading = datetime.datetime.now()
    try:
        driver.set_page_load_timeout(120)
        driver.get("https://www.aviasales.ru/search/{}".format(query))
    except WebDriverException:
        writeLog('Page loading timeout :' + query, 'goToPage')
    
    try:
        time.sleep(3)    
        wait_seconds = 600
        wait = WebDriverWait(driver, wait_seconds)
        wait.until(EC.invisibility_of_element((By.CLASS_NAME, "countdown")))
        loading_duration = (datetime.datetime.now() - start_loading).seconds
        printDelimiter('*')
        print("$$$$$   page loaded for {} seconds   $$$$$".format(loading_duration))
        time.sleep(1)
    except:
        print("\x1b[31m\"CAN'T WAIT THE PAGE LOAD ({} seconds)\"\x1b[0m".format(wait_seconds))
    

def closeExtraTab(driver): # В предположении что драйвер находится на рекламном окне после его открытия
    num_of_tabs = len(driver.window_handles)
    if num_of_tabs == 2:
        main_window_handle = None
        if driver.current_window_handle == driver.window_handles[0]:
            driver.switch_to_window(driver.window_handles[1])
            main_window_handle = driver.window_handles[0]
        elif driver.current_window_handle == driver.window_handles[1]:
            driver.switch_to_window(driver.window_handles[0])
            main_window_handle = driver.window_handles[1]
        title_of_closed_tab = driver.title
        driver.close()
        driver.switch_to_window(main_window_handle)
        writeLog("Extra tab *{}* has been closed".format(title_of_closed_tab), 'closeExtraTab')
    
    
def printDelimiter(char):
    delimiter = ''
    for i in range(118):
        delimiter += char
    print(delimiter)
    
    
def getCollection():
    client = MongoClient()
    db = client.aviasales_database # getting a database
    return db.ticket_collection

In [24]:
#hardcoded_year = None################

def processPage(driver, collection):
    tickets = driver.find_elements_by_class_name("ticket__wrapper")
    tickets = removeAdvertisingTickets(tickets)
    tickets = removeAbsentTickets(tickets)
    #print(tickets)
    print(driver.title[-9:], '\n')
    for ticket in tickets:
        collection.insert_one(processTicket(ticket))
        closeExtraTab(driver)
        
        
def removeAdvertisingTickets(tickets):
    for ticket in tickets:
        ticket_opener_text = ticket.find_element_by_class_name("ticket__opener").text
        if ticket_opener_text == 'Реклама':
            tickets.remove(ticket)
    return tickets


def removeAbsentTickets(tickets):
    for ticket in tickets:
        ticket_price = (ticket.find_element_by_class_name("buy-button__price").text).replace('\u2009', '')
        if ticket_price == '':
            tickets.remove(ticket)
    return tickets
        
        
def processTicket(ticket):
    ticket_price = int((ticket.find_element_by_class_name("buy-button__price-num").text).replace('\u2009', ''))
    
    ticket_opener = ticket.find_element_by_class_name("ticket__opener")
    ticket_opener.click()
    
    ticket_content = ticket.find_element_by_class_name("ticket__content")
    
    flights = ticket_content.find_elements_by_class_name("ticket__segment.segment")
    
    print("crawl_date_time: ", datetime.datetime.now())                    # debug
    print("price: ", ticket_price)                                         # debug
    
    post = dict()
    post['crawl_date_time'] = datetime.datetime.now()
    post['price'] = ticket_price
    post['flights'] = list()
    
    for flight in flights:
        post['flights'].append(processFlight(post, flight))
    
    return post

        
def processFlight(post, flight):
    flight_info = dict()
    flight_info['details'] = list()
    
    general_flight_info = getGeneralFlightInfo(flight)
    flight_info['city'] = general_flight_info[0]
    flight_info['date_time'] = general_flight_info[1]
    flight_info['total_time'] = general_flight_info[2]
    
    #nonlocal hardcoded_year
    #hardcoded_year = flight_general_info['date_time']['from'].year#############################
    
    
    detailed_flight_info = getDetailedFlightInfo(flight)
    for info in detailed_flight_info:
        flight_info['details'].append(info)
        
    flight_info_order = ['total_time', 'date_time', 'city']        # debug
    for info in flight_info_order:                                 # debug
        print('\t' + info + ": ", flight_info[info])               # debug
        
    print('\n')

    return flight_info
    
    
def getGeneralFlightInfo(flight):
# Общая информация
    flight_from = getGeneralPointInfo(flight, "segment-route__origin")

    flight_info_path = getGeneralPathInfo(flight)
    
    flight_to = getGeneralPointInfo(flight, "segment-route__destination")
    
    flight_info_city = {'from' : flight_from[0],
                        'to' : flight_to[0]}
    flight_info_date_time = {'from' : flight_from[1],
                             'to' : flight_to[1]}
    
    return flight_info_city, flight_info_date_time, flight_info_path


def getGeneralPathInfo(flight):
    flight_path = flight.find_element_by_class_name("segment-route__path")
    flight_total_time = re.split(' ', flight_path.find_element_by_class_name("segment-route__total-time").text)
    flight_total_time_info = getHoursAndMinutes(flight_total_time)
    return flight_total_time_info
    
    
def getHoursAndMinutes(time_string):
    hours = 0
    minutes = 0
    for word in time_string:
        if word[-1] == 'ч':
            hours = word[:-1]
        elif word[-1] == 'м':
            minutes = word[:-1]
    return {'hours' : hours, 'minutes' : minutes}
    
    
def getGeneralPointInfo(flight, segment_route):
    flight_point = flight.find_element_by_class_name(segment_route)
    flight_city = flight_point.find_element_by_class_name("segment-route__city").text
    time_point = re.split(':', flight_point.find_element_by_class_name("segment-route__time").text)
    date_point = re.split(' ', flight_point.find_element_by_class_name("segment-route__date").text)
    flight_date_time_point = datetime.datetime(int(date_point[2][:-1]), 
                                               translateDate(date_point[1]), 
                                               int(date_point[0]), 
                                               int(time_point[0]), 
                                               int(time_point[1]))
    return flight_city, flight_date_time_point

    
def translateDate(month):
    date_translator = {'янв' : 1, 'фев' : 2, 'мар' : 3, 'апр' : 4 ,'май' : 5 ,'июн' : 6, 'июл' : 7, 'авг' : 8,
                       'сен' : 9, 'окт' : 10, 'ноя' : 11, 'дек' : 12}
    return date_translator[month]
    
    
def getDetailedFlightInfo(flight):
    flight_info = list()
    try:
    # Детали перелета
        time.sleep(1)
    
        flight_itinerary = flight.find_element_by_class_name("segment__flights")

        # Flight legs
        flight_legs = flight_itinerary.find_elements_by_class_name("flight-leg")
        flight_legs_info = list()

        for flight_leg in flight_legs:
            flight_legs_info.append(getDetailedFlightLegInfo(flight_leg))


        # Flight stops    
        flight_stops = flight_itinerary.find_elements_by_class_name("flight-stop")
        flight_stops_info = list()

        for flight_stop in flight_stops:
            flight_stops_info.append(getDetailedFlightStopInfo(flight_stop))


        # Миксую через один инфу по остановкам и инфу по пересадкам (для восстановления хронологии в выводе)
        flight_info = mixFlightInfo(flight_legs_info, flight_stops_info)

        #for info in flight_info:                                               # debug
        #    if info['type'] == 'leg':                                          # debug
        #        print("\n\tflight date time: ", info['date_time'],             # debug
        #              "\n\t\tairport: ", info['airport'],                      # debug
        #              "\n\t\tairline: ", info['airline'],                      # debug
        #              "\n\t\tflight number: ", info['flight_number'])          # debug
        #    elif info['type'] == 'stop':                                       # debug
        #        print("\n\tstop time: ", info['time'],                         # debug
        #              "\n\tstop place: ", info['place'])                       # debug

        #print("\n")                                                        # debug
    except NoSuchElementException as exception:
        #print("\x1b[31m\"red\"\x1b[0m")
        print("\x1b[31m\"Element not found (getDetailedFlightInfo): ", exception, "\"\x1b[0m")
    return flight_info 
    
                                 
def mixFlightInfo(flight_legs_info, flight_stops_info):
    flight_info = [flight_stops_info[i//2] 
                   if i%2 else flight_legs_info[i//2] 
                   for i in range(len(flight_stops_info)*2+1)]
    return flight_info
                                 
    
def getDetailedFlightStopInfo(flight_stop):
    flight_attrs_tmp = dict()
    flight_attrs_tmp['place'] = flight_stop.find_element_by_class_name("flight-stop__stop-place").text
    flight_stop_time = re.split(' ', flight_stop.find_element_by_class_name("flight-stop__stop-time").text)
    flight_attrs_tmp['time'] = getHoursAndMinutes(flight_stop_time)
    flight_attrs_tmp['type'] = 'stop'
    return flight_attrs_tmp
    

def getDetailedFlightLegInfo(flight_leg):
    try:
        flight_attrs_tmp = dict()
        
        flight_attrs_tmp['type'] = 'leg'

        flight_time = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-time").text)
        flight_date = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-date").text)

        flight_date_time_from = getDetailedFlightLegDatetimeInfo(flight_time, flight_date, 'from')

        flight_date_time_to = getDetailedFlightLegDatetimeInfo(flight_time, flight_date, 'to')

        flight_attrs_tmp['date_time'] = {'from' : flight_date_time_from,
                                         'to' : flight_date_time_to}

        flight_attrs_tmp['airport'] = getDetailedFlightLegAirportInfo(flight_leg)

        flight_airline = flight_leg.find_element_by_class_name("flight-leg__airline-logo")
        flight_attrs_tmp['airline'] = flight_airline.find_element_by_css_selector('img').get_attribute('alt')
        
        flight_number = flight_leg.find_element_by_class_name("flight-leg__flight-info-link")
        flight_attrs_tmp['flight_number'] = flight_number.find_element_by_css_selector("span").text[19:]
    
    except NoSuchElementException as exception:
        print("\x1b[31m\"Element not found (getDetailedFlightLegInfo): ", exception, "\"\x1b[0m")
    return flight_attrs_tmp


def getDetailedFlightLegAirportInfo(flight_leg):
    flight_airport = flight_leg.find_element_by_class_name("flight-leg__flight-city")
    flight_airport_from = flight_airport.find_elements_by_tag_name('div')[0].text
    flight_airport_to = flight_airport.find_elements_by_tag_name('div')[1].text
    flight_airport_info = {'from' : 
                                   {'name' : flight_airport_from[4:],
                                    'iata' : flight_airport_from[:4]},
                                   'to' : 
                                   {'name' : flight_airport_to[4:],
                                    'iata' : flight_airport_to[:4]}}
    return flight_airport_info


def getDetailedFlightLegDatetimeInfo(flight_time, flight_date, point_key):
    point_index = -1
    if point_key == 'from':
        point_index = 0
    elif point_key == 'to':
        point_index = 1
    else:
        raise(ValueError)
    
    flight_time_point = re.split(':', flight_time[point_index])
    flight_date_point = re.split(' ', flight_date[point_index])

    #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
    flight_date_time_point = datetime.datetime(2019, #######hardcode######################################
                                               translateDate(flight_date_point[1][:-1]),
                                               int(flight_date_point[0]),
                                               int(flight_time_point[0]),
                                               int(flight_time_point[1])) 
    return flight_date_time_point


In [245]:
#query = 'MOW1307TNR20071'
#driver = createDriver(query)
#collection = getCollection()

In [98]:
#processPage(driver)

In [417]:
def printBeautifulTicket(post):
    print('_id: ', post['_id'])
    print('crawl_date_time: ', post['crawl_date_time'])
    print('flights: ')
    for flight in post['flights']:
        print('\tcity: ')
        print('\t\tfrom\t: ', flight['city']['from'])
        print('\t\tto\t: ', flight['city']['to'])
        print('\tdate_time: ')
        print('\t\tfrom\t: ', flight['date_time']['from'])
        print('\t\tto\t: ', flight['date_time']['to'])
        print('\tdetailes: ')
        for period in flight['details']:
            if period['type'] == 'leg':
                print('\t\tairline\t: ', period['airline'])
                print('\t\tairport: ')
                print('\t\t\tfrom: ')
                print('\t\t\t\tiata\t: ', period['airport']['from']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['from']['name'])
                print('\t\t\tto: ')
                print('\t\t\t\tiata\t: ', period['airport']['to']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['to']['name'])
                print('\t\tdate_time: ')
                print('\t\t\tfrom\t: ', period['date_time']['from'])
                print('\t\t\tto\t: ', period['date_time']['to'])
                print('\t\tflight_number\t: ', period['flight_number'])
                print('\t\ttype\t: ', period['type'])
            if period['type'] == 'stop':
                print('\t\tplace\t: ', period['place'])
                print('\t\ttime: ')
                print('\t\t\thours\t: ', period['time']['hours'])
                print('\t\t\tminutes\t: '), period['time']['minutes']
                
    print('\n\n')

In [419]:
for post in collection.find({'price':61765}):
    printBeautifulTicket(post)

_id:  5c60f9a9bee9fd0d3239381f
crawl_date_time:  2019-02-11 07:27:14.324000
flights: 
	city: 
		from	:  Москва
		to	:  Антананариву
	date_time: 
		from	:  2019-07-13 09:40:00
		to	:  2019-07-14 15:35:00
	detailes: 
		airline	:  Qatar Airways
		airport: 
			from: 
				iata	:  DME 
				name	:  Домодедово
			to: 
				iata	:  DOH 
				name	:  Доха
		date_time: 
			from	:  2019-07-13 09:40:00
			to	:  2019-07-13 14:50:00
		flight_number	:  QR-224
		type	:  leg
		place	:  Пересадка в Дохе
		time: 
			hours	:  2
			minutes	: 
		airline	:  Qatar Airways
		airport: 
			from: 
				iata	:  DOH 
				name	:  Доха
			to: 
				iata	:  NBO 
				name	:  Джомо Кеньятта
		date_time: 
			from	:  2019-07-13 17:40:00
			to	:  2019-07-13 23:35:00
		flight_number	:  QR-1341
		type	:  leg
		place	:  Пересадка в Найроби
		time: 
			hours	:  12
			minutes	: 
		airline	:  Kenya Airways
		airport: 
			from: 
				iata	:  NBO 
				name	:  Джомо Кеньятта
			to: 
				iata	:  TNR 
				name	:  Антананариву
		date_time: 

In [420]:
for post in collection.find({'price':61765}):
    pprint(post)

{'_id': ObjectId('5c60f9a9bee9fd0d3239381f'),
 'crawl_date_time': datetime.datetime(2019, 2, 11, 7, 27, 14, 324000),
 'flights': [{'city': {'from': 'Москва', 'to': 'Антананариву'},
              'date_time': {'from': datetime.datetime(2019, 7, 13, 9, 40),
                            'to': datetime.datetime(2019, 7, 14, 15, 35)},
              'details': [{'airline': 'Qatar Airways',
                           'airport': {'from': {'iata': 'DME ',
                                                'name': 'Домодедово'},
                                       'to': {'iata': 'DOH ', 'name': 'Доха'}},
                           'date_time': {'from': datetime.datetime(2019, 7, 13, 9, 40),
                                         'to': datetime.datetime(2019, 7, 13, 14, 50)},
                           'flight_number': 'QR-224',
                           'type': 'leg'},
                          {'place': 'Пересадка в Дохе',
                           'time': {'hours': 2, 'minutes': 50},
      

             {'city': {'from': 'Антананариву', 'to': 'Москва'},
              'date_time': {'from': datetime.datetime(2019, 7, 20, 3, 25),
                            'to': datetime.datetime(2019, 7, 21, 8, 0)},
              'details': [{'airline': 'Kenya Airways',
                           'airport': {'from': {'iata': 'TNR ',
                                                'name': 'Антананариву'},
                                       'to': {'iata': 'NBO ',
                                              'name': 'Джомо Кеньятта'}},
                           'date_time': {'from': datetime.datetime(2019, 7, 20, 3, 25),
                                         'to': datetime.datetime(2019, 7, 20, 6, 50)},
                           'flight_number': 'KQ-260',
                           'type': 'leg'},
                          {'place': 'Пересадка в Найроби',
                           'time': {'hours': 3, 'minutes': 10},
                           'type': 'stop'},
                     

In [433]:
collection.count_documents({'price':61765})

5

In [27]:
driver = webdriver.Firefox()
driver.get('https://www.aviasales.ru/search/MOW1307STO1')

In [25]:
def crawl_by_list(destinations, dates, driver): # dates - что то в духе ['1304','1404', ...]
    collection = getCollection()
    query = 'MOW'
    for date in dates:
        for destination in destinations:
            query = 'MOW' + date + destination + '1'
            goToPage(query, driver)
            processPage(driver, collection)

In [232]:
crawl_by_list(city_list_test, ['1304', '1805'], driver)

**********************************************************************************************************************
$$$$$page loaded for 14 seconds$$$$$
MOW → BRE
crawl_date_time:  2019-02-14 12:22:33.512006
price:  8156
	total_time:  {'minutes': '40', 'hours': '17'}
	date_time:  {'to': datetime.datetime(2019, 4, 14, 9, 40), 'from': datetime.datetime(2019, 4, 13, 17, 0)}
	city:  {'to': 'Бремен', 'from': 'Москва'}



crawl_date_time:  2019-02-14 12:22:35.382691
price:  8241
	total_time:  {'minutes': '5', 'hours': '15'}
	date_time:  {'to': datetime.datetime(2019, 4, 14, 10, 40), 'from': datetime.datetime(2019, 4, 13, 20, 35)}
	city:  {'to': 'Бремен', 'from': 'Москва'}



crawl_date_time:  2019-02-14 12:22:37.191753
price:  8241
	total_time:  {'minutes': '35', 'hours': '21'}
	date_time:  {'to': datetime.datetime(2019, 4, 14, 17, 10), 'from': datetime.datetime(2019, 4, 13, 20, 35)}
	city:  {'to': 'Бремен', 'from': 'Москва'}



crawl_date_time:  2019-02-14 12:22:39.039585
price:  8241
	t

StaleElementReferenceException: Message: The element reference of <div class="ticket__segment segment"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [ ]:
crawl_by_list(city_list, time_list[8:16], driver)

**********************************************************************************************************************
$$$$$   page loaded for 17 seconds   $$$$$
MOW → ALC 

crawl_date_time:  2019-02-18 09:10:40.281212
price:  7236
	total_time:  {'hours': '5', 'minutes': '20'}
	date_time:  {'to': datetime.datetime(2019, 3, 9, 13, 35), 'from': datetime.datetime(2019, 3, 9, 10, 15)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:10:43.942319
price:  8858
	total_time:  {'hours': '5', 'minutes': '15'}
	date_time:  {'to': datetime.datetime(2019, 3, 9, 12, 0), 'from': datetime.datetime(2019, 3, 9, 8, 45)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 27 seconds   $$$$$
MOW → AMS 

crawl_date_time:  2019-02-18 09:11:17.348132
price:  7340
	total_time:  {'hours': '9', 'minutes': '5'}
	date_time:  {'to': datetime.datetime

	total_time:  {'hours': '20', 'minutes': '45'}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 14, 30), 'from': datetime.datetime(2019, 3, 9, 19, 45)}
	city:  {'to': 'Антверпен', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:12:41.510126
price:  18127
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '20', 'minutes': '50'}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 9, 30), 'from': datetime.datetime(2019, 3, 9, 14, 40)}
	city:  {'to': 'Антверпен', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 24 seconds   $$$$$
MOW → BCN 

crawl_date_time:  2019-02-18 09:13:13.722228
price:  7148
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate 

crawl_date_time:  2019-02-18 09:14:42.542677
price:  13474
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '18', 'minutes': '10'}
	date_time:  {'to': datetime.datetime(2019, 3, 9, 20, 40), 'from': datetime.datetime(2019, 3, 9, 5, 30)}
	city:  {'to': 'Белфаст', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:14:46.852086
price:  13562
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-i

crawl_date_time:  2019-02-18 09:18:10.740443
price:  9570
	total_time:  {'hours': '16', 'minutes': 0}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 10, 50), 'from': datetime.datetime(2019, 3, 9, 20, 50)}
	city:  {'to': 'Бремен', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:18:13.356852
price:  9570
	total_time:  {'hours': '21', 'minutes': '20'}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 10, 50), 'from': datetime.datetime(2019, 3, 9, 15, 30)}
	city:  {'to': 'Бремен', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:18:15.828953
price:  11213
	total_time:  {'hours': '13', 'minutes': '50'}
	date_time:  {'to': datetime.datetime(2019, 3, 9, 16, 20), 'from': datetime.datetime(2019, 3, 9, 4, 30)}
	city:  {'to': 'Бремен', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:18:18.942140
price:  11213
	total_time:  {'hours': '13', 'minutes': '50'}
	date_time:  {'to': datetime.datetime(2019, 3, 9, 16, 20), 'from': datetime.datetime(2019, 3, 9, 4, 30)}
	city:  {'to': 'Бремен

crawl_date_time:  2019-02-18 09:19:40.837368
price:  23984
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '38', 'minutes': '40'}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 18, 25), 'from': datetime.datetime(2019, 3, 9, 5, 45)}
	city:  {'to': 'Брно', 'from': 'Москва'}


crawl_date_time:  2019-02-18 09:19:44.732089
price:  23984
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '38', 'minutes': '40'}
	date_time:  {'to': datetime.datetime(2019, 3, 10, 18, 25),

...

	total_time:  {'hours': '10', 'minutes': 0}
	date_time:  {'to': datetime.datetime(2019, 3, 11, 21, 45), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-18 11:53:21.663310
price:  7400
	total_time:  {'hours': '23', 'minutes': '10'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 10, 55), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-18 11:53:25.543932
price:  7400
	total_time:  {'hours': '26', 'minutes': '35'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 14, 20), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-18 11:53:30.886295
price:  7400
	total_time:  {'hours': '29', 'minutes': '35'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 17, 20), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  201

crawl_date_time:  2019-02-18 11:57:04.994857
price:  9896
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '17', 'minutes': '50'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 8, 20), 'from': datetime.datetime(2019, 3, 11, 16, 30)}
	city:  {'to': 'Валенсия', 'from': 'Москва'}


crawl_date_time:  2019-02-18 11:57:10.163471
price:  10658
	total_time:  {'hours': '7', 'minutes': 0}
	date_time:  {'to': datetime.datetime(2019, 3, 11, 23, 0), 'from': datetime.datetime(2019, 3, 11, 18, 0)}
	city:  {'to': 'Валенсия', 'from': 'Москва'}


crawl_date_time:  2019-02-18 11:57:14.873185
price:  10658
	total_time:  {'hours': '19', 'minutes': '30'}
	date_time:  {'to': datetime.dat

crawl_date_time:  2019-02-18 12:00:28.623275
price:  8842
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '9', 'minutes': '5'}
	date_time:  {'to': datetime.datetime(2019, 3, 11, 18, 0), 'from': datetime.datetime(2019, 3, 11, 10, 55)}
	city:  {'to': 'Венеция', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:00:32.487190
price:  8985
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '19', 'minutes': '40'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 10, 15), 'from': datetime.datetime(2019, 3, 11, 16, 35)}
	city:  {'to': 'Венеция', 'from': 'Москва'}


crawl_date_time:  201

**********************************************************************************************************************
$$$$$   page loaded for 58 seconds   $$$$$
MOW → HAJ 

crawl_date_time:  2019-02-18 12:05:09.562850
price:  6663
	total_time:  {'hours': '7', 'minutes': '35'}
	date_time:  {'to': datetime.datetime(2019, 3, 11, 22, 5), 'from': datetime.datetime(2019, 3, 11, 16, 30)}
	city:  {'to': 'Ганновер', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:05:13.450766
price:  7378
	total_time:  {'hours': '22', 'minutes': '45'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 10, 30), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Ганновер', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:05:19.368199
price:  7378
	total_time:  {'hours': '29', 'minutes': '5'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 16, 50), 'from': datetime.datetime(2019, 3, 11, 13, 45)}
	city:  {'to': 'Ганновер', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:05:23.931007
pric

"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '18', 'minutes': '35'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 8, 5), 'from': datetime.datetime(2019, 3, 11, 16, 30)}
	city:  {'to': 'Глазго', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:07:08.933374
price:  10480
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'hours': '19', 'minutes': '5'}
	date_time:  {'to': datetime.datetime(2019, 3, 12, 8, 5), 'from': datetime.datetime(2019, 3, 11, 16, 0)}
	city:  {'to': 'Глазго', 'from': 'Москва'}


crawl_date_time:  2019-02-18 12:07:12.497501
price:  10513
"Element not found (getD

In [7]:
city_list_test = {'BER':'Берлин', 'HAM':'Гамбург', 'BRE':'Бремен', }

#март
time_list = ['0103',
             '0203',
             '0303',
             '0403',
             '0503',
             '0603',
             '0703',
             '0803',
             '0903',
             '1003',
             '1103',
             '1203',
             '1303',
             '1403',
             '1503',
             '1603',
             '1703',
             '1803',
             '1903',
             '2003',
             '2103',
             '2203',
             '2303',
             '2403',
             '2503',
             '2603',
             '2703',
             '2803',
             '2903',
             '3003',
             '3103']

city_list = {'BER':'Берлин', 
             'HAM':'Гамбург', 
             'BRE':'Бремен', 
             'HAJ':'Ганновер', 
             'LEJ':'Лейпциг', 
             'DUS':'Дюссельдорф', 
             'CGN':'Кёльн', 
             'NUE':'Нюрнберг', 
             'MUC':'Мюнхен', 
             'PRG':'Прага', 
             'BRQ':'Брно', 
             'KLV':'Карловы Вары', 
             'VIE':'Вена', 
             'SZG':'Зальцбург', 
             'INN':'Инсбрук', 
             'BUD':'Будапешт', 
             'BRU':'Брюссель', 
             'ANR':'Антверпен', 
             'OST':'Брюгге', 
             'NCE':'Ницца', 
             'LYS':'Лион', 
             'PAR':'Париж', 
             'MPL':'Монпелье', 
             'SXB':'Страсбург', 
             'GVA':'Женева', 
             'ZRH':'Цюрих', 
             'BRN':'Берн',
             'MIL':'Милан',
             'ROM':'Рим',
             'VCE':'Венеция',
             'VRN':'Верона',
             'NAP':'Неаполь',
             'TRN':'Турин',
             'FLR':'Флоренция',
             'PMO':'Палермо',
             'BLQ':'Болонья',
             'PEG':'Перуджа',
             'AGP':'Малага',
             'BCN':'Барселона',
             'PMI':'Майорка',
             'MAD':'Мадрид',
             'ALC':'Аликанте',
             'SVQ':'Севилья',
             'VLC':'Валенсия',
             'QCN':'Таррагона',
             'IBZ':'Ибица',
             'LON':'Лондон',
             'MAN':'Манчестер',
             'EDI':'Эдинбург',
             'GLA':'Глазго',
             'CBG':'Кембридж',
             'BFS':'Белфаст',
             'DUB':'Дублин',
             'ORC':'Корк',
             'AMS':'Амстердам',
             'RTM':'Роттердам',
             'EIN':'Эйндховен',
             'LCA':'Ларнака',
             'ECN':'Никосия',
             'PFO':'Пафос',
             'STO':'Стокгольм',
             'GOT':'Гётерберг'
             }

city_list = OrderedDict(sorted(city_list.items(), key=lambda kv: kv[1]))

In [ ]:
date_translator = {'янв' : 1, 'фев' : 2, 'мар' : 3, 'апр' : 4 ,'май' : 5 ,'июн' : 6, 'июл' : 7, 'авг' : 8,
                   'сен' : 9, 'окт' : 10, 'ноя' : 11, 'дек' : 12}

for ticket in tickets:
        
    ticket_price = int((ticket.find_element_by_class_name("buy-button__price-num").text).replace('\u2009', ''))
    
    ticket_opener = ticket.find_element_by_class_name("ticket__opener")
    ticket_opener.click()
    
    ticket_content = ticket.find_element_by_class_name("ticket__content")
    
    flights = ticket_content.find_elements_by_class_name("ticket__segment.segment")
    
    print("crawl_date_time: ", datetime.datetime.now())
    print("price: ", ticket_price)
    
    post = dict()
    post['crawl_date_time'] = datetime.datetime.now()
    post['price'] = ticket_price
    post['flights'] = list()
    
    for flight in flights:
        
        # Общая информация
        flight_general_info = dict()
        
        flight_from = flight.find_element_by_class_name("segment-route__origin")
        flight_city_from = flight_from.find_element_by_class_name("segment-route__city").text
        time_from = re.split(':', flight_from.find_element_by_class_name("segment-route__time").text)
        date_from = re.split(' ', flight_from.find_element_by_class_name("segment-route__date").text)
        flight_date_time_from = datetime.datetime(int(date_from[2][:-1]), 
                                                  date_translator[date_from[1]], 
                                                  int(date_from[0]), 
                                                  int(time_from[0]), 
                                                  int(time_from[1]))
        
        
        flight_path = flight.find_element_by_class_name("segment-route__path")
        flight_total_time = re.split(' ', flight_path.find_element_by_class_name("segment-route__total-time").text)
        flight_general_info['total_time'] = {'hours' : int(flight_total_time[1][:-1]),
                                             'minutes' : int(flight_total_time[2][:-1])}
               
        flight_to = flight.find_element_by_class_name("segment-route__destination")
        flight_city_to = flight_to.find_element_by_class_name("segment-route__city").text
        time_to = re.split(':', flight_to.find_element_by_class_name("segment-route__time").text)
        date_to = re.split(' ', flight_to.find_element_by_class_name("segment-route__date").text)
        flight_date_time_to = datetime.datetime(int(date_to[2][:-1]), 
                                                date_translator[date_to[1]], 
                                                int(date_to[0]), 
                                                int(time_to[0]), 
                                                int(time_to[1]))
        flight_general_info['city'] = {'from' : flight_city_from,
                                       'to' : flight_city_to}
        flight_general_info['date_time'] = {'from' : flight_date_time_from,
                                            'to' : flight_date_time_to}
        
        
        flight_general_info['details'] = list()
        
        
        # Детали перелета
        flight_itinerary = flight.find_element_by_class_name("segment__flights")
        flight_legs = flight_itinerary.find_elements_by_class_name("flight-leg")
        flight_legs_info = list()
        
        for flight_leg in flight_legs:
            flight_attrs_tmp = dict()
            flight_time = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-time").text)
            flight_time_from = re.split(':', flight_time[0])
            flight_time_to = re.split(':', flight_time[1])
            flight_date = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-date").text)
            flight_date_from = re.split(' ', flight_date[0])
            flight_date_to = re.split(' ', flight_date[1])
            #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
            flight_date_time_from = datetime.datetime(flight_general_info['date_time']['from'].year, 
                                                                   date_translator[flight_date_from[1][:-1]],
                                                                   int(flight_date_from[0]),
                                                                   int(flight_time_from[0]),
                                                                   int(flight_time_from[1])) 
            #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
            flight_date_time_to = datetime.datetime(flight_general_info['date_time']['from'] .year, 
                                                                   date_translator[flight_date_to[1][:-1]],
                                                                   int(flight_date_to[0]),
                                                                   int(flight_time_to[0]),
                                                                   int(flight_time_to[1]))
            flight_attrs_tmp['date_time'] = {'from' : flight_date_time_from,
                                             'to' : flight_date_time_to}
            
            flight_airports = flight_leg.find_element_by_class_name("flight-leg__flight-city")
            flight_airport_from = flight_airports.find_elements_by_tag_name('div')[0].text
            flight_airport_to = flight_airports.find_elements_by_tag_name('div')[1].text
            flight_attrs_tmp['airport'] = {'from' : 
                                           {'name' : flight_airport_from[4:],
                                            'iata' : flight_airport_from[:4]},
                                           'to' : 
                                           {'name' : flight_airport_to[4:],
                                            'iata' : flight_airport_to[:4]}}
            
            flight_airline = flight_leg.find_element_by_class_name("flight-leg__airline-logo")
            flight_attrs_tmp['airline'] = flight_airline.find_element_by_css_selector('img').get_attribute('alt')
            flight_number = flight_leg.find_element_by_class_name("flight-leg__flight-info-link")
            flight_attrs_tmp['flight_number'] = flight_number.find_element_by_css_selector("span").text[19:]
            #print("((((((((((((((", flight_attrs_tmp['airline'], ")))))))))))))))))))))")
            flight_attrs_tmp['type'] = 'leg'
            flight_legs_info.append(flight_attrs_tmp)
        
        flight_stops = flight_itinerary.find_elements_by_class_name("flight-stop")
        flight_stops_info = list()
        
        for flight_stop in flight_stops:
            flight_attrs_tmp = dict()
            flight_attrs_tmp['place'] = flight_stop.find_element_by_class_name("flight-stop__stop-place").text
            flight_stop_time = re.split(' ', flight_stop.find_element_by_class_name("flight-stop__stop-time").text)
            flight_attrs_tmp['time'] = {'hours' : int(flight_stop_time[0][:-1]),
                                        'minutes' : int(flight_stop_time[1][:-1])}
            flight_attrs_tmp['type'] = 'stop'
            flight_stops_info.append(flight_attrs_tmp)
        
        
        flight_general_info_order = ['total_time', 'date_time', 'city']
        for info in flight_general_info_order:
            print('\t' + info + ": ", flight_general_info[info])
        
        # Миксую через один инфу по остановкам и инфу по пересадкам (для восстановления хронологии в выводе)
        flight_info = [flight_stops_info[i//2] 
                       if i%2 else flight_legs_info[i//2] 
                       for i in range(len(flight_stops_info)*2+1)]
        
        for info in flight_info:
            if info['type'] == 'leg':
                print("\n\tflight date time: ", info['date_time'],
                      "\n\t\tairport: ", info['airport'],
                      "\n\t\tairline: ", info['airline'],
                      "\n\t\tflight number: ", info['flight_number'])
            elif info['type'] == 'stop':
                print("\n\tstop time: ", info['time'],
                      "\n\tstop place: ", info['place'])
        
        print("\n\n\n")
        
        
        for info in flight_info:
            flight_general_info['details'].append(info)
            
        post['flights'].append(flight_general_info)
    
    collection.insert_one(post)
        